In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
DATA_FILE_PATH = os.path.join(os.getcwd(),'data','100_50')

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\Nate\\Documents\\GitHub\\SOundstuff\\Top-Op\\MachineLerning\\Agents\\100_50'